### Proovedores

#### Importaciones y lectura de objetos

In [1]:
import pandas as pd

In [2]:
LFA1 = pd.read_excel(r"./Data/Active Analysis/LFA1.xlsx", dtype='string')
LFB1 = pd.read_excel(r"./Data/Active Analysis/LFB1.xlsx", dtype='string')

Cambiar headers

In [3]:
new_headers = LFA1.iloc[0]  
LFA1.columns = new_headers  
LFA1 = LFA1.iloc[1:]  
new_headers = LFB1.iloc[0]  
LFB1.columns = new_headers  
LFB1 = LFB1.iloc[1:]  


Division Activa/Obsoleta

In [4]:
LFB1_active = LFB1[LFB1['SOC_STAT'] == 'Activa']
LFB1_obsoleta = LFB1[LFB1['SOC_STAT'] == 'Obsoleta']

In [5]:
print("No. filas en LFB1 activa --->  ",LFB1_active.shape[0])
print("No. filas LFB1 obsoleta  --->  ",LFB1_obsoleta.shape[0])

No. filas en LFB1 activa --->   82278
No. filas LFB1 obsoleta  --->   35285


LFB1 activa/inactiva SPERR Bloqueo Contabilidad para sociedad

In [6]:
LFB1_activa_bloqueada = LFB1_active[LFB1_active['SPERR'] == 'X']
LFB1_obsoleta_bloqueada = LFB1_obsoleta[LFB1_obsoleta['SPERR'] == 'X']

In [7]:
print("No. filas en LFB1 activa con SPERR bloqueada --->  ",LFB1_activa_bloqueada.shape[0])
print("No. filas LFB1 obsoleta con SPERR bloqueada --->  ",LFB1_obsoleta_bloqueada.shape[0])

No. filas en LFB1 activa con SPERR bloqueada --->   36572
No. filas LFB1 obsoleta con SPERR bloqueada --->   4848


Agrupacion por LIFNR, al menos una activa

In [8]:
contains_activa = LFB1.groupby('LIFNR')['SOC_STAT'].transform(lambda x: (x == 'Activa').any())

LFB1_LIFNIR_sin_ninguna_activa = LFB1[~contains_activa]
LFB1_LIFNR_minimo_una_activa = LFB1[contains_activa]

In [11]:
print("No. de filas en LFB1 sin ninguna 'Activa' en su grupo LIFNR --->  ", LFB1_LIFNIR_sin_ninguna_activa.shape[0])
print("No. de filas en LFB1 con al menos una 'Activa' en su grupo LIFNR --->  ", LFB1_LIFNR_minimo_una_activa.shape[0])

No. de filas en LFB1 sin ninguna 'Activa' en su grupo LIFNR --->   16211
No. de filas en LFB1 con al menos una 'Activa' en su grupo LIFNR --->   101352


In [10]:
LFB1_LIFNIR_unica_sin_ninguna_activa =  LFB1_LIFNIR_sin_ninguna_activa.drop_duplicates(subset=['LIFNR'])
LFB1_LIFNR_unica_minimo_una_activa = LFB1_LIFNR_minimo_una_activa.drop_duplicates(subset=['LIFNR'])

In [12]:
print("No. de filas en LFB1 sin ninguna 'Activa' en su grupo LIFNR unica--->  ", LFB1_LIFNIR_unica_sin_ninguna_activa.shape[0])
print("No. de filas en LFB1 con al menos una 'Activa' en su grupo LIFNR unica --->  ", LFB1_LIFNR_unica_minimo_una_activa.shape[0])

No. de filas en LFB1 sin ninguna 'Activa' en su grupo LIFNR unica--->   11695
No. de filas en LFB1 con al menos una 'Activa' en su grupo LIFNR unica --->   50134


Cruce LFA1 | LFB1 con LIFNR agrupado sin actividad y sin marcados para borrado

In [35]:
LFA1_sin_marcados_borrado = LFA1[LFA1["LOEVM"].isna()]

In [47]:
df_inner_join_obsolete_mb = pd.merge(LFA1_sin_marcados_borrado, LFB1_LIFNIR_unica_sin_ninguna_activa[['LIFNR', 'SOC_STAT']], on='LIFNR', how='inner')

In [48]:
print("No. de filas Cruce LFA1 | LFB1 con LIFNR agrupado sin actividad y sin marcados para borrado --->  ", df_inner_join_obsolete_mb.shape[0])

No. de filas Cruce LFA1 | LFB1 con LIFNR agrupado sin actividad y sin marcados para borrado --->   11094


In [50]:
df_inner_join_obsolete_mb.to_excel(r'./Data/Active Analysis/inner_join_Complete_mb.xlsx', index=False, sheet_name="Sheet1")

Cruce LFA1 | LFB1 con LIFNR agrupado sin actividad y sin bloqueo central contabilizacion

In [43]:
LFA1_sin_bloqueo_cont = LFA1[LFA1["SPERR"].isna()]

In [45]:
df_inner_join_obsolete_bc = pd.merge(LFA1_sin_bloqueo_cont, LFB1_LIFNIR_unica_sin_ninguna_activa[['LIFNR', 'SOC_STAT']], on='LIFNR', how='inner')

In [51]:
df_inner_join_obsolete_bc.to_excel(r'./Data/Active Analysis/inner_join_Complete_bc.xlsx', index=False, sheet_name="Sheet1")

In [52]:
print("No. de filas Cruce LFA1 sin bloqueos contabilidad| LFB1 con LIFNR agrupado sin actividad  --->  ", df_inner_join_obsolete_bc.shape[0])

No. de filas Cruce LFA1 sin bloqueos contabilidad| LFB1 con LIFNR agrupado sin actividad  --->   11208
